In [1]:
import pandas as pd
import requests
from config import data as userdata
import os
import time
import numpy as np

# 使用 API 取得實時資料

In [2]:
def getRealTimeData(city='bj', datatype='airquality', day='2018-04-01',force=False):
    datadir = './realTimeData'
    if not os.path.isdir(datadir):
        os.mkdir(datadir)
    url = 'https://biendata.com/competition/' + datatype + '/' + city +  '/' + day + '-0/' + day + '-23/2k0d1d8'
    filename = datadir + "/" + city + '_' + datatype + '_' + day + '.csv'
    if not force and os.path.exists(filename):
        return filename
    responses= requests.get(url)
    if responses.text[0] != '{':
        with open(filename,'w') as f:
            f.write(responses.text)
        return filename
    else:
        print('error:\n',responses.text)
        return ''

In [3]:
name = getRealTimeData(day='2018-04-13')
print(name)
bjdata = pd.read_csv(name)
bjdata.head(5)

./realTimeData/bj_airquality_2018-04-13.csv


,id,station_id,time,PM25_Concentration,PM10_Concentration,NO2_Concentration,CO_Concentration,O3_Concentration,SO2_Concentration
0,2957584,dongsi_aq,2018-04-13 00:00:00,44.0,120.0,74.0,1.7,20.0,32.0
1,2957585,tiantan_aq,2018-04-13 00:00:00,47.0,107.0,76.0,1.8,13.0,16.0
2,2957586,guanyuan_aq,2018-04-13 00:00:00,41.0,110.0,73.0,1.6,27.0,26.0
3,2957587,wanshouxigong_aq,2018-04-13 00:00:00,38.0,128.0,69.0,1.8,13.0,21.0
4,2957588,aotizhongxin_aq,2018-04-13 00:00:00,41.0,120.0,78.0,1.7,25.0,24.0


# 取得之前資料

In [3]:
previousBjData = pd.read_csv('./beijing_17_18_aq.csv')
previousLdData = pd.read_csv('./London_historical_aqi_forecast_stations_20180331.csv')

In [4]:
def getPreviousData(thisday, nday=2, city='bj'):
    formatDate = "%Y-%m-%d"
    thisday = time.mktime(time.strptime(thisday, formatDate))

    alldays = list()
    if city == 'bj':
        formatDate = '%Y-%m-%d %H:%M:%S'
        pData = previousBjData
        column = ['test_id','time','PM2.5','PM10','O3']
    elif city == 'ld':
        formatDate = '%Y/%m/%d %H:%M'
        pData = previousLdData
        column = ['test_id','time','PM2.5','PM10']

    for i in range(24*nday):
        tmp = thisday + i*60*60
        if city == 'ld':
            tmp = time.localtime(tmp)
            days = '%d/%d/%d %d:00' % (tmp.tm_year, tmp.tm_mon, tmp.tm_mday, tmp.tm_hour)
        else:
            days = time.strftime(formatDate, time.localtime(tmp))
        alldays.append(days)
    
    #print(alldays)
    
    return pData.loc[pData['time'].isin(alldays),column]

In [5]:
tmp = getPreviousData('2017-04-20',city='bj')
tmp.head()

,test_id,time,PM2.5,PM10,O3
2431,aotizhongxin_aq,2017-04-20 02:00:00,131.0,NaN,26.0
2432,aotizhongxin_aq,2017-04-20 03:00:00,121.0,NaN,53.0
2433,aotizhongxin_aq,2017-04-20 04:00:00,33.0,73.0,77.0
2434,aotizhongxin_aq,2017-04-20 05:00:00,43.0,80.0,94.0
2435,aotizhongxin_aq,2017-04-20 06:00:00,41.0,135.0,98.0


# 輸出答案

要預測未來兩天 48小時 的 PM2.5 PM10 O3 

> 我想 4/11 送答案就是預測 4/12 ~ 4/13 的空氣品質

總共有 48 個點的數值需要預測

每個點要有 48 個小時的資料

站點包含 北京與倫敦 的點

id 格式 站點名#hours

In [37]:
sample = pd.read_csv('sample_submission.csv')
print(sample.shape)

sample_id = sample['test_id']
stations = list()
for id in sample_id.iloc[:]:
    [station, hours] = id.split('#')
    if station not in stations:
        stations.append(station)
print(len(stations), ' stations')

(2304, 4)
48  stations


In [38]:
realdata = pd.read_csv(getRealTimeData(day='2018-04-13'))
realdata = realdata['station_id']
bj_stations = list()
for station in realdata.iloc[:]:
    if station not in bj_stations:
        bj_stations.append(station)

print(len(bj_stations), ' bj stations')
#print(bj_stations)

realdata = pd.read_csv(getRealTimeData(city='ld',day='2018-04-13'))
realdata = realdata['station_id']
ld_stations = list()
for station in realdata.iloc[:]:
    if station not in ld_stations:
        ld_stations.append(station)

print(len(ld_stations), ' ld stations')
#print(ld_stations)

35  bj stations
19  ld stations


In [6]:
def injectAnswer(sample, fromDate, answer, formatDate = "%Y-%m-%d %H:%M:%S", column=['station_id', 'time', 'PM25_Concentration', 'PM10_Concentration', 'O3_Concentration']):
    #formatDate = "%Y-%m-%d %H:%M:%S"
    fromDate = time.mktime(time.strptime(fromDate, formatDate))
    [rows, cols] = answer.shape
    for i in range(rows):
        data = answer.iloc[i]
        thisDate = time.mktime(time.strptime(data[column[1]],formatDate))
        thisDate = int((thisDate - fromDate) / (60*60))
        thisid = data[column[0]] + '#' + str(thisDate)
        sample.loc[sample['test_id'] == thisid, 'PM2.5'] = data[column[2]]
        sample.loc[sample['test_id'] == thisid, 'PM10'] = data[column[3]]
        if len(column) > 4:
            sample.loc[sample['test_id'] == thisid, 'O3'] = data[column[4]]

def makeSubmissions(fromDate, force=False):
    sample = pd.read_csv('sample_submission.csv')
    sample.loc[:, 'PM2.5'] = np.nan
    sample.loc[:, 'PM10'] = np.nan
    sample.loc[:, 'O3'] = np.nan
    
    formatDate = "%Y-%m-%d %H:%M:%S"
    fromDate_t = time.strptime(fromDate,formatDate)
    
    firstDate = time.strftime("%Y-%m-%d", fromDate_t)
    fromDate_t = time.localtime(time.mktime(fromDate_t) + (24*60*60))
    secondDate = time.strftime("%Y-%m-%d", fromDate_t)
    
    realdata = pd.read_csv(getRealTimeData(city='bj',day=firstDate, force=force))
    injectAnswer(sample, fromDate, realdata)

    realdata = pd.read_csv(getRealTimeData(city='bj',day=secondDate, force=force))
    injectAnswer(sample, fromDate, realdata)

    realdata = pd.read_csv(getRealTimeData(city='ld',day=firstDate, force=force))
    injectAnswer(sample, fromDate, realdata)

    realdata = pd.read_csv(getRealTimeData(city='ld',day=secondDate, force=force))
    injectAnswer(sample, fromDate, realdata)
        
    return sample


def makePreviousData(today=''):
    sample = pd.read_csv('sample_submission.csv')
    sample.loc[:, 'PM2.5'] = np.nan
    sample.loc[:, 'PM10'] = np.nan
    sample.loc[:, 'O3'] = np.nan
    
    if today == '':
        today = time.gmtime(time.time()+24*60*60)
        #today.tm_year -= 1
        #today = time.strftime("%Y-%m-%d", today)
        today_bj = '%d-%d-%d' % (today.tm_year-1, today.tm_mon, today.tm_mday)
        today_ld = '%d/%d/%d' % (today.tm_year-1, today.tm_mon, today.tm_mday)
        today = '%d-%d-%d' % (today.tm_year-1, today.tm_mon, today.tm_mday)
    
    #print(today)
    previousdata = getPreviousData(today, city='bj')
    injectAnswer(sample, today_bj + ' 00:00:00', previousdata, column=previousdata.columns)
    
    previousdata = getPreviousData(today, city='ld')
    injectAnswer(sample, today_ld + ' 00:00', previousdata, formatDate = '%Y/%m/%d %H:%M', column=previousdata.columns)
        
    return sample, today
        
def generateAnswer(answer, method='', desc=''):
    # remove negative
    answer.loc[answer['PM2.5'] <= 0, 'PM2.5'] = np.nan
    answer.loc[answer['PM10'] <= 0, 'PM10'] = np.nan
    answer.loc[answer['O3'] <= 0, 'O3'] = np.nan
    
    if method == 'mean':
        answer = answer.fillna(answer.mean())
    elif method == 'pad':
        answer = answer.fillna(method=method)
    
    todayDate = time.localtime(time.time())
    todayDate = time.strftime("%m%d", todayDate)
    
    answer.to_csv('./answer.csv', index=False, header=True)
    
    datadir = './answer'
    if not os.path.isdir(datadir):
        os.mkdir(datadir)
    
    filename = datadir + '/answer' + todayDate + method
    if len(desc) > 0:
        filename = filename + '_' +desc
    filename = filename + '.csv'
    answer.to_csv(filename, index=False, header=True)
    
    return answer,filename

In [13]:
fromDate = '2018-04-21 00:00:00'

answer_origin = makeSubmissions(fromDate, force=True)
answer_origin.head()

,test_id,PM2.5,PM10,O3
0,dongsi_aq#0,124.0,NaN,55.0
1,dongsi_aq#1,92.0,NaN,71.0
2,dongsi_aq#2,93.0,NaN,72.0
3,dongsi_aq#3,NaN,NaN,NaN
4,dongsi_aq#4,85.0,NaN,49.0


In [14]:
answer_origin.loc[0,'PM10'] = 7.0
answer_origin.head()

,test_id,PM2.5,PM10,O3
0,dongsi_aq#0,124.0,7.0,55.0
1,dongsi_aq#1,92.0,NaN,71.0
2,dongsi_aq#2,93.0,NaN,72.0
3,dongsi_aq#3,NaN,NaN,NaN
4,dongsi_aq#4,85.0,NaN,49.0


In [16]:
answer,filename = generateAnswer(answer_origin, method='pad',desc='2122')
print(filename)
answer.head()

./answer/answer0423pad_2122.csv


,test_id,PM2.5,PM10,O3
0,dongsi_aq#0,124.0,7.0,55.0
1,dongsi_aq#1,92.0,7.0,71.0
2,dongsi_aq#2,93.0,7.0,72.0
3,dongsi_aq#3,93.0,7.0,72.0
4,dongsi_aq#4,85.0,7.0,49.0


In [17]:
[previous, pToday] = makePreviousData()
print(pToday)
previous.to_csv('./previous.csv')
previous.head()

2017-4-24


,test_id,PM2.5,PM10,O3
0,dongsi_aq#0,NaN,NaN,NaN
1,dongsi_aq#1,34.0,39.0,82.0
2,dongsi_aq#2,23.0,98.0,103.0
3,dongsi_aq#3,19.0,NaN,115.0
4,dongsi_aq#4,19.0,270.0,118.0


In [18]:
previous_combine = previous.combine_first(answer_origin)
previous_combine.to_csv('./previous_cobine_api.csv')
previous_combine.head()

,test_id,PM2.5,PM10,O3
0,dongsi_aq#0,124.0,7.0,55.0
1,dongsi_aq#1,34.0,39.0,82.0
2,dongsi_aq#2,23.0,98.0,103.0
3,dongsi_aq#3,19.0,NaN,115.0
4,dongsi_aq#4,19.0,270.0,118.0


In [20]:
answer, filename = generateAnswer(previous_combine, method='pad', desc='lastyear0423-0424 combine 0420-0421')
print(filename)
answer.head()

./answer/answer0423pad_lastyear0423-0424 combine 0420-0421.csv


,test_id,PM2.5,PM10,O3
0,dongsi_aq#0,124.0,7.0,55.0
1,dongsi_aq#1,34.0,39.0,82.0
2,dongsi_aq#2,23.0,98.0,103.0
3,dongsi_aq#3,19.0,98.0,115.0
4,dongsi_aq#4,19.0,270.0,118.0


# 送出答案

In [21]:
from api_submit import sendSubmission

print(sendSubmission(answerfile=filename, name='answer.csv', desc=filename))

{"success": true}


# 思考策略

先考慮如何建立模型

輸入是歷史資料, 輸出則是兩天的未來預測

考慮到訓練參數出來

把一年的歷史資料 分成訓練與測試



# SMAPE

計分方式

In [114]:
import math

def SMAPE(actual=list(), forecast=list()):
    if len(actual)!=len(forecast):
        return 2
    
    total = 0
    total_len = 0
    for i in actual.index:
        if math.isnan(actual[i]) or math.isnan(forecast[i]):
            continue
        tmp = (abs(forecast[i]) + abs(actual[i]))/2
        if tmp != 0:
            tmp = abs(forecast[i] - actual[i]) / tmp
        total += tmp
        total_len += 1
    
    #print(total,',',total_len)
    
    if total_len == 0:
        answer = 0
    else:
        answer = total / total_len
        
    return answer

def smape(actual, predicted):
    dividend= np.abs(np.array(actual) - np.array(predicted))
    denominator = np.abs(np.array(actual)) + np.abs(np.array(predicted))
    
    return 2 * np.mean(np.divide(dividend, denominator, out=np.zeros_like(dividend), where=denominator!=0, casting='unsafe'))

In [56]:
SMAPE([1,2,3],[1,1,1])

0.5555555555555555

In [69]:
smape([1.0,2.0,3.0], [1.0,1.0,1.0])

0.55555555555555547

In [78]:
actualData = makeSubmissions('2018-04-15 00:00:00',force=True)

In [110]:
actualData2 = actualData.dropna(axis=0,how='any')

(1165, 4)


In [116]:
actualData2 = actualData.dropna(axis=0,how='any')
forecastData = pd.read_csv('./answer/answer0414mean.csv')
forecastData = forecastData.fillna(0)
forecastData = forecastData.loc[actualData2.index,:]

a1 = SMAPE(actualData2['PM2.5'].loc[:],forecastData['PM2.5'].loc[:])
a2 = SMAPE(actualData2['PM10'].loc[:],forecastData['PM10'].loc[:])
a3 = SMAPE(actualData2['O3'].loc[:],forecastData['O3'].loc[:])

print('PM2.5 :',a1)
print('PM10 :',a2)
print('O3 :',a3)

PM2.5 : 0.693784236971
PM10 : 0.758121816806
O3 : 0.715826464322
